In [ ]:
import os
import pandas as pd

In [30]:
def merge_csv_files_in_directory(directory,output):
    dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                filepath = os.path.join(root, file)
                df = pd.read_csv(filepath)
                dfs.append(df)
    merged_df = dfs[0]
    
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on=['ID_RIO', 'coord_TRAMO'], how='inner')
    
    print(merged_df)
    print("Ahora guardamos en local")
    merged_df.to_csv(output, index=False)

In [31]:
def verificar_permisos_directorio(directory):
    # Verificar si el directorio existe
    if not os.path.exists(directory):
        print(f"El directorio {directory} no existe.")
        return False
    
    # Verificar permisos de lectura
    if not os.access(directory, os.R_OK):
        print(f"No tienes permisos de lectura para el directorio {directory}.")
        return False
    
    # Verificar permisos de escritura
    if not os.access(directory, os.W_OK):
        print(f"No tienes permisos de escritura para el directorio {directory}.")
        return False
    
    # Verificar permisos de ejecución
    if not os.access(directory, os.X_OK):
        print(f"No tienes permisos de ejecución para el directorio {directory}.")
        return False
    
    print(f"Tienes permisos completos para acceder al directorio {directory}.")
    return True

In [33]:
# UNIR TABLAS EN UNA SOLA
# Directorios de vertientes
vertientes_directorios = ['/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO', '/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/MEDITERRANEO']
output_file = ["/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO/ContaminacionAtlantico.csv","/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/MEDITERRANEO/ContaminacionMediterraneo.csv"]

# Iterar sobre cada vertiente
for vertiente in vertientes_directorios:
    verificar_permisos_directorio(vertiente)
    for salida in output_file:
        merged_df = merge_csv_files_in_directory(vertiente,salida)

Tienes permisos completos para acceder al directorio /Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ATLANTICO.
          ID_RIO                  coord_TRAMO  valor_Fosfato  valor_Nitrato  \
0          10034  (823091.6071, 4736100.5283)        0.00560         1.0605   
1          10034  (807702.0344, 4738426.8291)        0.00459         1.5600   
2          10034    (805142.39, 4751389.2164)        0.01748         1.1355   
3          10034    (805142.39, 4751389.2164)        0.01748         1.1355   
4       10034996  (803804.7411, 4750624.8265)        0.01748         1.1355   
...          ...                          ...            ...            ...   
325643  10099922  (267441.0379, 3990306.2624)        0.02500         0.7325   
325644  10099922   (267173.9276, 3989732.971)        0.02500         0.7325   
325645  10099924   (269586.4743, 3991970.456)        0.02500         0.5660   
325646  10099924  (269126.9504, 3991275.7842)        0.02500         0.5660  

In [ ]:
#CONCATENAR FICHEROS DE LAS VERTIENTS
output = "/Users/silviadelatorre/Desktop/TFG/PFG/Results/VALORES CONTAMINACION/ValoresContaminación.csv"
